Импортируем нужные библиотеки

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymorphy2
import pymystem3
import re
import pickle
from tqdm import tqdm

Прочитаем файл и очистим текст (для примера используются только первые 1000 строк файла)

In [ ]:
def clean_text(text: str):
    return re.sub(r'[^A-Za-z0-9А-Яа-яЁё]+', ' ', text)

lines = []

with open('JOB_LIST.csv', 'r', encoding="utf-8", errors='ignore') as f:
    for i in range(1000):
        lines.append(clean_text(f.readline()).strip())
    lines = lines[1:]
        
lines[:3]

Пропишем, как будет определяться ключевое слово, по которому будем группировать. В нашем случае будет выделяться нормальная форма "главного" существительного, которое первым по порядку встречается в строке 

In [ ]:
morph = pymorphy2.MorphAnalyzer()

def lemm_and_noun(word: str):
    parsed = morph.parse(word)
    is_noun = True
    if len(parsed) > 1:
        is_noun = 'ADJF' not in parsed[1].tag
    return parsed[0].normal_form, ('NOUN' in parsed[0].tag) and is_noun

Создадим словарь, где ключами являются нормализованные "главные" существительные, а значениями - полные названия профессий из файла

In [ ]:
dct = {}

for line in tqdm(lines):
    for word in line.split():
        norm, is_noun = lemm_and_noun(word)
        if is_noun:
            dct[norm] = dct.get(norm, []) + [line]

In [ ]:
dct

Итоговый словарь является результатом группировки похожих профессий